In [317]:
import numpy as np

# can be 'PIN' or 'ROLLER'
class Joint(object):
    def __init__(self, id=0, x=0, y=0):
        self.id=id
        self.x=x
        self.y=y

def calculate_distance(start_joint, end_joint):
    dx = end_joint.x - start_joint.x
    dy = end_joint.y - start_joint.y
    return np.abs(dx*dx + dy*dy)
# list of joints: dictionary {ID : {Joint}  }, 
# list of edges: tuples (A, B)    
    
class Truss(object):
    def __init__(self, edgeList=[], 
                 joints=[], 
                 fixed_joint='', 
                 roller_joint='', 
                 center_joint='', 
                 dict_of_symmetric_joints={}, 
                 reduced_joint_list =[]):
        self.edgeList = edgeList
        self.joints = joints
        self.fixed_joint = fixed_joint
        self.roller_joint = roller_joint
        self.center_joint = center_joint
        self.dict_of_symmetric_joints = dict_of_symmetric_joints        
        self.reduced_joint_list = reduced_joint_list

        self.member_cross_area = 19.625e-4 #m^2
        self.member_density = 8050 #kg / m^3
        self.joint_mass = 5 #kg
        self.jointDict = {}
        
        
        for joint in self.joints:
            self.jointDict[joint.id] = joint
        
    def calculate_mass(self):
        total_length = 0
        num_joints = len(self.joints)
        for edge in self.edgeList:
            joint_1 = self.jointDict[edge[0]]
            joint_2 = self.jointDict[edge[1]]
            length = calculate_distance(joint_1, joint_2)
            total_length += length
        mass = total_length * self.member_cross_area * self.member_density
        mass += num_joints * self.joint_mass
        return mass

    def print_truss(self):
        for joint in self.joints:
            print(joint.id, joint.x, joint.y)
        for edge in self.edgeList:
            print(edge)
    
    def print_joints(self):
        for joint in self.joints:
            print(joint.id, joint.x, joint.y)
    def print_edges(self):
        for edge in self.edgeList:
            print(edge)
    
    def update_joint(self, joint_id, x, y):
        new_joint = Joint(joint_id, x, y)
        self.jointDict[joint_id] = new_joint
        for i in range(len(self.joints)):
            if self.joints[i].id == joint_id:
                self.joints[i] = new_joint
                return
            

In [318]:
# truss solving function:
# takes in:
# TRUSS object (list of nodes, list of edges)
# list of forces
# AppliedForces : (JointID, magnitude, direction (in y direction))
def solve_forces(appliedForces, truss):
    numAppliedForces = len(appliedForces)
    numJoints = len(truss.joints)
    numReactions = 3 # 2 (Fx, Fy) at fixed joint, 1 (Fy) at roller
    numEdges = len(truss.edgeList)
    numEquations = numAppliedForces + numEdges + numReactions
    M = np.zeros((numEquations, numEquations))
    #M system of equations to solve [<applied forces> .... <x forces in joints> <y forces in joints> ]
    #F : vector of forces to solve for [<applied forces>, <forces in beams>, <reactions>]
    
    E = np.zeros((numEquations, 1)) #solution to system of equations M*F = E
    #populate rows for  E 
    for i in range(numAppliedForces):
        E[i] = appliedForces[i][1]
        M[i][i] = 1

    i = 0 
    for i in range(numJoints):
        joint = truss.joints[i]
        jointId = joint.id
        row = i*2 + numAppliedForces
        for j in range(len(appliedForces)):
            col = j
            if(appliedForces[j][0] == jointId):
                M[row][col] = 1 # Force:(jointId, magnitude)
        
        for j in range(len(truss.edgeList)):
            col = j+numAppliedForces
            edge = truss.edgeList[j]
            otherJointId = None
            if(edge[0] == jointId):
                otherJointId = edge[1]
            elif(edge[1] == jointId):
                otherJointId = edge[0]
            if(otherJointId != None):
                otherJoint = truss.jointDict[otherJointId]
                x1 = joint.x
                y1 = joint.y
                x2 = otherJoint.x
                y2 = otherJoint.y
                dx = x2 - x1
                dy = y2 - y1
                
                theta = np.abs(np.arctan(dy/dx))
                x_multiplier = np.sign(dx) * np.cos(theta)
                y_multiplier = np.sign(dy) * np.sin(theta)
                
                M[row][col] = y_multiplier
                M[row+1][col] = x_multiplier
        if(joint.id == truss.fixed_joint):
            M[row][numEquations - 3] = 1 # reaction, assume in positive direction
            M[row + 1][numEquations - 2] = 1 #y reaction, assume in positive direction
        if(joint.id == truss.roller_joint):
            M[row][numEquations - 1] = 1
    
    F = np.linalg.solve(M, E)
    F = np.around(F,2)
    return M, E, F

In [319]:
#joint_x_list = [0, 0.5, 1]
#joint_y_list = [0, 0.866, 0]
#jointIds = ['A', 'B', 'C']
joint_x_list = [0, 1, 2, 3, 4]
joint_y_list = [0, 1, 0, 1, 0]
jointIds = ['A', 'B', 'C', 'D', 'E']
jointList = []
for i in range(len(jointIds)):
    joint = Joint(jointIds[i],joint_x_list[i],joint_y_list[i]);
    jointList.append(joint)

edge_list = [('A', 'B'), ('B', 'C'), ('A', 'D'), ('D', 'E'), ('D', 'B'), ('E','C'), ('B', 'E')]

truss = Truss(edge_list, jointList, 'A', 'E')
mass = truss.calculate_mass()
print(mass)

562.13625


In [320]:
applied_force_list = [('B', -100)] # 500 Newtons
M, E, F = solve_forces(applied_force_list, truss)
print(M)
print(F)

[[ 1.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.70710678  0.          0.31622777  0.          0.          0.
   0.          1.          0.          0.        ]
 [ 0.          0.70710678  0.          0.9486833   0.          0.          0.
   0.          0.          1.          0.        ]
 [ 1.         -0.70710678 -0.70710678  0.          0.          0.          0.
  -0.31622777  0.          0.          0.        ]
 [ 0.         -0.70710678  0.70710678  0.          0.          1.          0.
   0.9486833   0.          0.          0.        ]
 [ 0.          0.          0.70710678  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.70710678  0.          0.          0.          1.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.31622777 -0.70710678  0.          0.
   0.          0.  

In [359]:
import operator

def sort_by_x(list_of_nodes):
    list_of_nodes.sort(key = operator.attrgetter('x'))
    
def getKey(item):
    return item[1]

def refine(x, y, std):
    x_refined = np.random.normal(x, std)
    y_refined = np.random.normal(y, std)
    return np.round(x_refined,2), np.round(y_refined,2)

def sorted_list_of_nodes_by_distances(joint_list, start_joint):
    distance_list = []
    for joint in joint_list:
        if(joint.id == start_joint.id):
            continue
        r = calculate_distance(start_joint, joint)
        distance_list.append((joint.id, r))
    return sorted(distance_list, key = getKey)

class TrussGenerator:
    def __init__(self, width, max_height, min_height, min_edge_length):
        self.width = width
        self.x_min = -width/2
        self.x_max = width/2
        self.max_height = max_height
        self.min_height = min_height
        self.min_edge_length = min_edge_length
        self.ascii_offset = 65
        self.joint_list = []
        self.dict_of_symmetric_joints = {}
        self.center_joint_id = ''
        self.pin_joint_id = ''
        self.roller_joint_id = ''
        self.list_of_edges = []
        self.reduced_joint_list = []
    
    def reset(self):
        self.joint_list = []
        self.dict_of_symmetric_joints = {}
        self.center_joint_id = ''
        self.pin_joint_id = ''
        self.roller_joint_id = ''
        self.list_of_edges = []
        self.reduced_joint_list = []
    
    def add_joint_to_list(self, joint_list, x, y):
        index = len(joint_list)
        joint = Joint(chr(index + self.ascii_offset), x, y)
        joint_list.append(joint)
        return joint.id

    def add_symmetric_pair_of_joints(self, id1, id2):
        self.dict_of_symmetric_joints[id1] = id2
        self.dict_of_symmetric_joints[id2] = id1
        self.reduced_joint_list.append(id1)

    def generate_nodes(self, num_joints):
        joint_list = []
        index = 0
        x_end = width/2
        x_start = -width/2
        # place fixed joints 
        self.pin_joint_id = self.add_joint_to_list(joint_list, x_start, 0)
        self.roller_joint_id = self.add_joint_to_list(joint_list, x_end, 0)
        self.add_symmetric_pair_of_joints(self.pin_joint_id, self.roller_joint_id)
        # place central joints
        is_even = False
        num_central_joints = 1
        if(num_joints % 2 == 0):
            is_even = True
            num_central_joints = 2
        #place first central joint
        y_coarse = np.random.uniform(self.min_height,self.max_height)
        y_refined = np.random.normal(y_coarse, min_edge_length/2)
        self.center_joint_id = self.add_joint_to_list(joint_list, 0,y_refined)
        self.add_symmetric_pair_of_joints(self.center_joint_id, self.center_joint_id)
        if(is_even):
            y_coarse_2 = np.random.uniform(min_height, max_height)
            if(y_coarse_2 == y_coarse):
                y_coarse_2 = (y_coarse + 1) % (self.max_height - self.min_height) + self.min_height
            y_refined_2 = np.random.normal(y_coarse, min_edge_length/2)
            center2_id = self.add_joint_to_list(joint_list, 0,y_refined_2)
            self.add_symmetric_pair_of_joints(center2_id, center2_id)

        min_length = self.min_edge_length
        x_grid_size = (self.width / 2) / self.min_edge_length - 2
        y_grid_size = (self.max_height - self.min_height) / self.min_edge_length       
        
        total_positions = x_grid_size * y_grid_size
        if(total_positions < num_joints):
            min_length = min_length / 2
        
        #generate list of tuples with all possible positions
        possible_position_list = []
        for x in np.arange(min_length, width/2 - min_length, min_length):
            for y in np.arange(min_height, max_height, min_length):
                possible_position_list.append((x, y))
        
        num_remaining_joints = int((num_joints - 2 - num_central_joints) / 2)
        std = self.min_edge_length / 4
        for i in range(num_remaining_joints):
            list_size = len(possible_position_list)
            index = np.random.randint(0,list_size)
            x = possible_position_list[index][0]
            y = possible_position_list[index][1]
            x, y = refine(x,y, std)
            id1 = self.add_joint_to_list(joint_list, x, y)
            possible_position_list.pop(index)
            # mirror the joint
            x_mirrored = -x
            id2 = self.add_joint_to_list(joint_list, x_mirrored, y)
            self.add_symmetric_pair_of_joints(id1, id2)

        self.joint_list = joint_list
        return joint_list
    
    def connect_nodes(self, id1, id2):
        self.connected_node_table[id1].append(id2)
        self.connected_node_table[id2].append(id1)
        edge = (id1, id2)
        self.list_of_edges.append(edge)

    def build_oonnections(self):
        sort_by_x(self.joint_list)
        
        self.list_of_edges = []
        self.connected_node_table = {}
        #initialize table with empty lists
        for joint in self.joint_list:
            self.connected_node_table[joint.id] = []
        
        num_nodes = len(self.joint_list)
        num_desired_connections = 2*num_nodes - 3
        if(num_desired_connections < 3):
            return
        even_num = False
        if(num_nodes % 2 == 0):
            even_num = True
        
        index_start = 0
        index_end = index_end = int((num_nodes - 1)/2)
        if(even_num):
            index_end = int(num_nodes - 2)/2
        
        index_center_node_start = int(index_end)
        index_center_node_end = int(index_end + 1)
        if(even_num):
            index_center_node_end = int(index_center_node_start + 2)
        
        num_connections = 0
        subset_of_nodes = self.joint_list[0:index_center_node_end]
        
        
        if(even_num): #connect 2 center joints
            joint1_id = self.joint_list[index_center_node_start].id
            joint2_id = self.joint_list[index_center_node_start+1].id
            self.connect_nodes(joint1_id, joint2_id)
        else: #connect 2 joints closest to center joint
            joint1_id = self.joint_list[index_center_node_start - 1].id
            joint2_id = self.dict_of_symmetric_joints[joint1_id]
            self.connect_nodes(joint1_id, joint2_id)

        for node in subset_of_nodes:
            list_of_nodes_by_distance = sorted_list_of_nodes_by_distances(self.joint_list, node)
            closest_node_id = list_of_nodes_by_distance[0][0]
            next_closest_node_id = list_of_nodes_by_distance[1][0]
            # check if closest node already connected
            if(closest_node_id not in self.connected_node_table[node.id]):
                self.connect_nodes(node.id, closest_node_id)
                #mirror the connection
                mirror_id_1 = self.dict_of_symmetric_joints[node.id]
                mirror_id_2 = self.dict_of_symmetric_joints[closest_node_id]
                self.connect_nodes(mirror_id_1, mirror_id_2)
            # check if next closest node is already connected 
            if(next_closest_node_id not in self.connected_node_table[node.id]):
                self.connect_nodes(node.id, next_closest_node_id)
                #mirror the connection
                mirror_id_1 = self.dict_of_symmetric_joints[node.id]
                mirror_id_2 = self.dict_of_symmetric_joints[next_closest_node_id]
                self.connect_nodes(mirror_id_1, mirror_id_2)
        
        succesful = (len(self.list_of_edges) == num_desired_connections) 

        return self.list_of_edges, self.connected_node_table, succesful

    def generate_truss(self, num_joints):
        self.reset()
        self.generate_nodes(num_joints)
        self.build_oonnections()
        
        return Truss(self.list_of_edges, 
                     self.joint_list, 
                     self.pin_joint_id, 
                     self.roller_joint_id, 
                     self.center_joint_id,
                     self.dict_of_symmetric_joints, 
                     self.reduced_joint_list)

In [360]:
# test truss generator
width = 4
max_height = 1
min_height = 0
min_edge_length = 0.5
truss_generator = TrussGenerator(width, max_height, min_height, min_edge_length)
joint_list = truss_generator.generate_nodes(3)
#list_of_connections, connected_node_table, succesful = build_oonnections(list_of_joints, dict_of_symmetric_joints, center_joint_id, pin_joint_id)
for joint in joint_list:
    print(joint.id, joint.x, joint.y)


A -2.0 0
B 2.0 0
C 0 0.9013205805300835


In [361]:
# test for build_connections function
list_of_connections, connected_node_table, succesful = truss_generator.build_oonnections()
print(succesful)
print("connections \n", list_of_connections)
print("table \n", connected_node_table)

True
connections 
 [('A', 'B'), ('A', 'C'), ('B', 'C')]
table 
 {'A': ['B', 'C'], 'C': ['A', 'B'], 'B': ['A', 'C']}


In [362]:
# test generate_truss()
num_joints = 5
truss = truss_generator.generate_truss(num_joints)
for edge in truss.edgeList:
    print(edge)
    
for joint in truss.joints:
    print(joint.id, joint.x, joint.y)

('E', 'D')
('A', 'E')
('B', 'D')
('A', 'C')
('B', 'C')
('E', 'C')
('D', 'C')
A -2.0 0
E -0.54 0.48
C 0 0.28533301606270256
D 0.54 0.48
B 2.0 0


In [363]:
def evaluate_fitness(truss):
    center_joint = truss.center_joint
    default_force = -100 #kN
    default_member_strength = 490 # kN, for radius of 2.5 cm
    
    applied_forces = [(center_joint, default_force)]
    M, E, F = solve_forces(applied_forces, truss)
    max_force_in_member = F.argmax()
    
    safety_factor = default_member_strength / max_force_in_member
    mass = truss.calculate_mass()
    # maybe should calculate maximum mass?
    fitness = safety_factor / mass
    return fitness

In [364]:
# test fitness function
fitness = evaluate_fitness(truss)
print(fitness)

0.475865840808


In [369]:
import copy

def calculate_dx_dy(r, theta):
    dx = r * np.cos(theta)
    dy = r * np.sin(theta)
    return dx, dy

class Constraints:
    def __init__(self,  width, max_height, min_height, min_edge_length, min_number_joints, max_number_joints):
        self.width = width
        self.max_height = max_height
        self.min_height = min_height
        self.min_edge_length = min_edge_length
        self.min_number_joints = min_number_joints
        self.max_number_joints = max_number_joints

class MutationParameters:
    def __init__(self, num_iterations_per_joint, num_joint_mutation_directions, min_r = 0.2, max_r = 0.5):
        self.num_joint_mutation_directions = num_joint_mutation_directions
        self.num_iterations_per_joint = num_iterations_per_joint
        self.min_r = min_r
        self.max_r = max_r

class Population:
    def generate_population(self):
        start = self.constraints.min_number_joints
        end = self.constraints.max_number_joints + 1
        truss_generator = TrussGenerator(self.constraints.width, 
                                         self.constraints.max_height, 
                                         self.constraints.min_height, 
                                         self.constraints.min_edge_length)
        for num_joints in range(start, end):
            new_truss = truss_generator.generate_truss(num_joints)
            self.truss_list.append(new_truss)
            
    def __init__(self, constraints):
        self.constraints = constraints
        self.truss_list = []
        self.mutation_parameters = MutationParameters(1, 4)
        self.generate_population()
    
    # generate children for a single truss
    def generate_children(self, truss, number_children):
        pass
    
    def mutate_single_joint(truss, joint_id):
        pass
    
    def generate_single_joint_mutations(self, truss, joint_id, num_directions, min_r, max_r):
        list_of_children = []
        if(num_directions < 1):
            print("error, num_directions cannot be less than 1")
        # construct table
        angle_increment = 2*np.pi / num_directions
        direction_table = []
        for i in range(0, num_directions):
            start_angle = i * angle_increment
            end_angle = (i+1) * angle_increment
            direction_table.append((start_angle, end_angle))
        
        #alternately use a uniform distribution
        for direction in direction_table:
            r = np.random.uniform(min_r, max_r)
            spread = angle_increment
            target_direction = direction[0]
            theta = np.random.normal(target_direction, angle_increment)
            
            new_child = copy.deepcopy(truss)
            joint = new_child.jointDict[joint_id]
            dx, dy = calculate_dx_dy(r, theta)
            x_new = joint.x + dx
            y_new = joint.y + dy
            symmetric_joint_id = new_child.dict_of_symmetric_joints[joint_id]
            
            new_child.update_joint(joint_id, x_new, y_new)
            new_child.update_joint(symmetric_joint_id, x_new, y_new)
            list_of_children.append(new_child)    
        
        return list_of_children    
    
    def add_children_to_population(self, list_of_children):
        for child in list_of_children:
            self.truss_list.append(child)
        
    def spawn_children_single_joint_mutation(self, truss):
        for joint_id in truss.reduced_joint_list:
            if(joint_id == truss.fixed_joint or joint_id == truss.roller_joint):
                continue
            for i in range(self.mutation_parameters.num_iterations_per_joint):
                new_children = self.generate_single_joint_mutations(truss, 
                                                               joint_id, 
                                                               self.mutation_parameters.num_joint_mutation_directions,
                                                               self.mutation_parameters.min_r,
                                                               self.mutation_parameters.max_r)
                self.add_children_to_population(new_children)
                
    def print_population(self):
        count = 0
        for truss in self.truss_list:
            print("child no. ", count)
            truss.print_joints()
            count +=1
            print("\n")
        
        

In [370]:
# Test generate population
width = 4
max_height = 1
min_height = 0
min_edge_length = 0.5
min_number_joints = 3
max_number_joints = 5

constraints = Constraints(width,max_height,min_height, min_edge_length, min_number_joints, max_number_joints)
population = Population(constraints)
for truss in population.truss_list:
    truss.print_joints()
    print(len(truss.reduced_joint_list))

A -2.0 0
C 0 0.8389988966916999
B 2.0 0
2
A -2.0 0
C 0 0.41088124135732385
D 0 0.5981969738931434
B 2.0 0
3
A -2.0 0
E -0.97 0.11
C 0 0.8659296787847438
D 0.97 0.11
B 2.0 0
3


In [371]:
# Test mutate single joints in truss
children = population.generate_single_joint_mutations(population.truss_list[0], 'A', 4, 0.1, 0.5)
count = 1
for child in children:
    print("truss no. ", count)
    child.print_joints()
    count = count+1
    print("\n")

truss no.  1
A -2.01950426472 -0.310307908875
C 0 0.8389988966916999
B -2.01950426472 -0.310307908875


truss no.  2
A -1.99893357945 0.438009930043
C 0 0.8389988966916999
B -1.99893357945 0.438009930043


truss no.  3
A -2.20641071563 -0.0296022708055
C 0 0.8389988966916999
B -2.20641071563 -0.0296022708055


truss no.  4
A -2.11267341141 0.138082916675
C 0 0.8389988966916999
B -2.11267341141 0.138082916675




In [372]:
# test spawn children
print("before spawning children")
population.print_population()

population.spawn_children_single_joint_mutation(population.truss_list[0])
print("after spawning children")
population.print_population()

before spawning children
child no.  0
A -2.0 0
C 0 0.8389988966916999
B 2.0 0


child no.  1
A -2.0 0
C 0 0.41088124135732385
D 0 0.5981969738931434
B 2.0 0


child no.  2
A -2.0 0
E -0.97 0.11
C 0 0.8659296787847438
D 0.97 0.11
B 2.0 0


[<__main__.Truss object at 0x000002679136E3C8>, <__main__.Truss object at 0x000002679136E208>, <__main__.Truss object at 0x00000267914B7278>, <__main__.Truss object at 0x00000267914B7400>]
after spawning children
child no.  0
A -2.0 0
C 0 0.8389988966916999
B 2.0 0


child no.  1
A -2.0 0
C 0 0.41088124135732385
D 0 0.5981969738931434
B 2.0 0


child no.  2
A -2.0 0
E -0.97 0.11
C 0 0.8659296787847438
D 0.97 0.11
B 2.0 0


child no.  3
A -2.0 0
C -0.165605499679 0.593008068422
B 2.0 0


child no.  4
A -2.0 0
C -0.234313236683 1.1478943553
B 2.0 0


child no.  5
A -2.0 0
C 0.315041360085 0.924934616267
B 2.0 0


child no.  6
A -2.0 0
C -0.313372051679 0.859328933283
B 2.0 0


